In [ ]:
'''
Instructions to run the benchmarks:

Use Python 3 to run this script
For SymEngine benchmarks compile with -DWITH_TCMALLOC=yes -DWITH_FLINT=yes -DINTEGER_CLASS=flint
For GiNaC benchmarks compile with the same CXX flags as SymEngine
Move (or symlink) all the benchmarks from symengine/symengine repo and symengine/symengine.py repo to this folder
'''

from subprocess import Popen, PIPE, TimeoutExpired
import json
import os
import signal

test1 = ["expand2", list(range(4, 50, 2))]
test2 = ["expand6", list(range(100, 1100, 100))]
test4 = ["expand6b", list(range(100, 1100, 100))]
test3 = ["expand7", list(range(10, 50, 3))]
test5 = ["symengine_bench", list(range(10, 41, 2))]

tests = [test1, test2, test3, test4, test5]
for test in tests:
    name = test[0]
    param = test[1][:]
    for b in range(6):
        lib = ["./%s %s","./%s_ginac %s", "./%s.m %s", "./%s.py %s", "maple -q %s.mpl -D n=%s", "./%s_piranha %s"][b]
        time = [float('inf')] * len(test[1])
        for i in range(len(param)):
            test_name = lib % (name, param[i])
            command = 'LD_PRELOAD=/usr/lib/libtcmalloc.so '+test_name
            with Popen(command, shell=True, stdout=PIPE, preexec_fn=os.setsid) as process:
                try:
                    output = process.communicate(timeout=120)[0]
                    #print(output, command)
                    time[i] = float(output.splitlines()[0].decode('utf-8').replace("ms", "").strip())
                    time[i] = max(0.01, time[i])
                except TimeoutExpired:
                    os.killpg(process.pid, signal.SIGINT) # send signal to the process group
                    break
            #print(test_name, time[i])
        test.append(time)
        print(time)
with open('data.json', 'w') as f:
    f.write(json.dumps(tests, indent=2))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import json
import numpy as np
from matplotlib.ticker import FormatStrFormatter
with open('data.json', 'r') as f:
    tests = json.loads(f.read())
for test in tests:
    name = test[0]
    param = test[1][:]
    for i in range(2, len(test)):
        test[i] = np.array(test[i])/1000 # convert from ms to s
    if (len(test) > 4):
        plt.loglog(param, test[2], 'go-', label="SymEngine")
        plt.loglog(param, test[3], 'bo-', label="GiNaC")
        plt.loglog(param, test[4], 'ro-', label="Mathematica")
        plt.loglog(param, test[5], 'yo-', label="SymPy")
        plt.loglog(param, test[6], 'co-', label="Maple")
    else:
        plt.loglog(param, test[2], 'go-', label="SymEngine")
        plt.loglog(param, test[3], 'yo-', label="SymPy")
        
    if (len(test) > 7):
        plt.loglog(param, test[-1], 'mo-', label="Piranha")
        
    plt.xlim([param[0]*0.9, param[-1]*1.1])
    plt.xlabel('N')
    plt.ylabel('Time [s]')
    plt.title("Benchmark "+name)
    lgd = plt.legend(loc='center right', bbox_to_anchor=(1.4, 0.5))
    plt.gca().xaxis.grid(which="both")
    plt.gca().yaxis.grid()
    plt.tick_params(axis='x', which='minor')
    plt.gca().xaxis.set_minor_formatter(FormatStrFormatter("%.0f"))
    plt.show()